In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def create_motion_psf(image_size, motion_angle=45, motion_dist=15):
    """Create Point Spread Function for motion blur."""
    psf = np.zeros(image_size)
    center = (image_size[0]//2, image_size[1]//2)

    angle_rad = np.deg2rad(motion_angle)
    dx = motion_dist * np.cos(angle_rad)
    dy = motion_dist * np.sin(angle_rad)

    x1, y1 = center[1] - dx//2, center[0] - dy//2
    x2, y2 = center[1] + dx//2, center[0] + dy//2

    cv2.line(psf, (int(x1), int(y1)), (int(x2), int(y2)), 1, 1)
    return psf / psf.sum()

def motion_degradation(image, psf, noise_variance):
    """Apply motion blur and add noise to image."""
    # Apply motion blur
    freq = np.fft.fft2(image)
    freq_kernel = np.fft.fft2(psf, s=image.shape)
    blurred = np.real(np.fft.ifft2(freq * freq_kernel))

    # Add noise
    noise = np.random.normal(0, np.sqrt(noise_variance), image.shape)
    degraded = blurred + noise

    return np.clip(degraded, 0, 255).astype(np.uint8)

def inverse_filter(degraded, psf, threshold=0.01):
    """Apply inverse filtering with threshold."""
    freq_degraded = np.fft.fft2(degraded)
    freq_kernel = np.fft.fft2(psf, s=degraded.shape)

    # Avoid division by very small values
    mask = np.abs(freq_kernel) > threshold
    freq_restored = np.zeros_like(freq_degraded, dtype=complex)
    freq_restored[mask] = freq_degraded[mask] / freq_kernel[mask]

    restored = np.real(np.fft.ifft2(freq_restored))
    return np.clip(restored, 0, 255).astype(np.uint8)

def wiener_filter(degraded, psf, K):
    """Apply Wiener filter restoration."""
    freq_degraded = np.fft.fft2(degraded)
    freq_kernel = np.fft.fft2(psf, s=degraded.shape)

    freq_kernel_conj = np.conjugate(freq_kernel)
    freq_kernel_mag2 = np.abs(freq_kernel)**2

    freq_restored = freq_kernel_conj / (freq_kernel_mag2 + K) * freq_degraded
    restored = np.real(np.fft.ifft2(freq_restored))

    return np.clip(restored, 0, 255).astype(np.uint8)

def process_and_visualize(image, noise_variances=[650, 65, 6.5]):
    """Process image with different noise levels and restoration methods."""
    # Motion blur parameters
    psf = create_motion_psf(image.shape, motion_angle=45, motion_dist=15)

    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    plt.suptitle('Motion Blur Removal using Inverse and Wiener Filters', fontsize=16)

    for i, variance in enumerate(noise_variances):
        # Create degraded image
        degraded = motion_degradation(image, psf, variance)

        # Restore using inverse filter
        inverse_restored = inverse_filter(degraded, psf, threshold=0.01)

        # Restore using Wiener filter
        K = variance / 255.0  # Adjust K based on noise variance
        wiener_restored = wiener_filter(degraded, psf, K)

        # Display results
        axes[i, 0].imshow(degraded, cmap='gray')
        axes[i, 0].set_title(f'Degraded (σ²={variance})')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(inverse_restored, cmap='gray')
        axes[i, 1].set_title('Inverse Filter')
        axes[i, 1].axis('off')

        axes[i, 2].imshow(wiener_restored, cmap='gray')
        axes[i, 2].set_title('Wiener Filter')
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

def main():
    # Create or load a test image
    image = cv2.imread('your_image.jpg', cv2.IMREAD_GRAYSCALE)
    if image is None:
        # Create test image with text
        image = np.zeros((200, 400), dtype=np.uint8)
        cv2.putText(image, 'Digital', (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, 255, 2)
        cv2.putText(image, 'Image', (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 2, 255, 2)
        cv2.putText(image, 'Processing', (50, 180), cv2.FONT_HERSHEY_SIMPLEX, 2, 255, 2)

    # Process with different noise levels
    process_and_visualize(image, noise_variances=[650, 65, 6.5])

if __name__ == "__main__":
    main()

Output hidden; open in https://colab.research.google.com to view.